# ET-287 - Processamento de sinais usando redes neurais

_Aluno: Denys Derlian Carvalho Brito_

---

## Projeto Exame - Classificação de Desempenho em Criptografia Quântica com Redes Neurais Profundas

**Objetivo:** Desenvolver um classificador baseado em redes neurais profundas capaz de prever, a partir de variáveis heterogêneas de um ambiente de criptografia quântica, a classe de desempenho (“Ótimo” vs. “Subótimo”) com alta acurácia e boa capacidade de generalização.


No contexto da computação quântica, a criptografia quântica oferece novos paradigmas de segurança baseados em princípios de mecânica quântica. Sob essa perspectiva, a aplicação de técnicas de aprendizado de máquina tem sido explorada para otimizar e aprimorar o desempenho dos sistemas de criptografia quântica. Todavia, a complexidade inerente a esses sistemas, aliada à heterogeneidade dos dados gerados, impõe desafios significativos para a construção de modelos preditivos eficazes. Nesse cenário, propõe uma abordagem que foca na classificação da **performance da rede** de forma integrada, considerando não apenas os parâmetros quânticos e de segurança, mas também métricas de performance de rede, características de _big data_ e utilização de recursos computacionais, que em conjunto determinam a viabilidade prática dos sistema em cenários operacionais reais.

---

## 5. Algoritmos de ML clássico para comparação: _Random Forest, SVM e XGBoost_

In [1]:
# Importart bibliotecas a partir do módulo config
from modules.config import *

/home/denysderlian/anaconda3/envs/NNenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-19 01:05:49.230880: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-19 01:05:49.742258: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-19 01:05:51.892921: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see sl

Num GPUs Available:  1


In [2]:
# Carregar os dados pre-processados
df_preprocessed = pd.read_csv("./data/preprocessed/df_preprocessed.csv")
X_numerical_df = pd.read_csv("./data/preprocessed/X_numerical_preprocessed.csv")
X_categorical_df = pd.read_csv("./data/preprocessed/X_categorical_preprocessed.csv")
y = pd.read_csv("./data/preprocessed/y_preprocessed.csv")


TARGET_VARIABLE = 'Performance_Target'
CATEGORICAL_COLUMNS = df_preprocessed.select_dtypes(include=['object', 'category']).columns.tolist()
NUMERICAL_COLUMNS = df_preprocessed.select_dtypes(include=['number']).columns.difference([TARGET_VARIABLE]).tolist()

In [3]:
X_dev_idx_ml, X_test_idx_ml = train_test_split(
        np.arange(len(y)), 
        test_size=0.2, 
        stratify=y.values.ravel(), 
        random_state=RANDOM_SEED
    )

X_num_dev_ml = X_numerical_df.iloc[X_dev_idx_ml].values
X_cat_dev_ml = X_categorical_df.iloc[X_dev_idx_ml].values
X_num_test_ml = X_numerical_df.iloc[X_test_idx_ml].values
X_cat_test_ml = X_categorical_df.iloc[X_test_idx_ml].values
y_dev_ml = y.iloc[X_dev_idx_ml].values.ravel().astype(int)
y_test_ml = y.iloc[X_test_idx_ml].values.ravel().astype(int)
print(f"✓ Split criado: Dev={len(y_dev_ml)}, Test={len(y_test_ml)}")

# Aplicar scaling no dev set e transformar test set
scaler_ml = MinMaxScaler()
X_num_dev_scaled_ml = scaler_ml.fit_transform(X_num_dev_ml)
X_num_test_scaled_ml = scaler_ml.transform(X_num_test_ml)

# Concatenar features numéricas (escaladas) + categóricas (label-encoded)
X_dev_ml = np.hstack([X_num_dev_scaled_ml, X_cat_dev_ml])
X_test_ml = np.hstack([X_num_test_scaled_ml, X_cat_test_ml])

print(f"\n📊 Dados preparados para modelos clássicos:")
print(f"  Dev set: {X_dev_ml.shape} | Test set: {X_test_ml.shape}")
print(f"  Features: {X_num_dev_scaled_ml.shape[1]} numéricas + {X_cat_dev_ml.shape[1]} categóricas")
print(f"  Classes dev: {np.bincount(y_dev_ml)}")
print(f"  Classes test: {np.bincount(y_test_ml)}")

✓ Split criado: Dev=800, Test=200

📊 Dados preparados para modelos clássicos:
  Dev set: (800, 28) | Test set: (200, 28)
  Features: 18 numéricas + 10 categóricas
  Classes dev: [734  66]
  Classes test: [184  16]


In [4]:
def evaluate_classic_model(model, model_name, X_train, y_train, X_test, y_test):
    """
    Treina e avalia um modelo clássico de ML
    """
    print(f"\n{'─'*70}")
    print(f"🔄 Treinando {model_name}...")
    print(f"{'─'*70}")
    
    # Treinar
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Predições
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    
    # Encontrar threshold ótimo para F1
    prec, rec, thr = precision_recall_curve(y_test, y_pred_prob)
    f1_scores = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-8)
    best_thr = thr[np.argmax(f1_scores)]
    
    y_pred = (y_pred_prob >= best_thr).astype(int)
    
    # Métricas
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    pr_auc = average_precision_score(y_test, y_pred_prob)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    accuracy = accuracy_score(y_test, y_pred) 
    recall = recall_score(y_test, y_pred, zero_division=0)
    cm = confusion_matrix(y_test, y_pred)
    
    # Resultados
    print(f"✓ Treinamento concluído em {train_time:.2f}s")
    print(f"  Threshold otimizado: {best_thr:.4f}")
    print(f"\n📊 Métricas no Test Set:")
    print(f"  ROC-AUC:   {roc_auc:.4f}")
    print(f"  PR-AUC:    {pr_auc:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"\nConfusion Matrix:")
    print(cm)
    
    return {
        'model_name': model_name,
        'train_time': train_time,
        'threshold': best_thr,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy,
        'confusion_matrix': cm
    }

print("✓ Função de avaliação definida")

✓ Função de avaliação definida


In [5]:
# Configuração para otimização Optuna dos modelos clássicos
OPTUNA_CLASSIC_N_TRIALS = 50  # Número de trials para cada modelo
OPTUNA_CLASSIC_CV_SPLITS = 5  # Stratified K-Fold splits para validação

print(f"⚙️ Configuração Optuna para Modelos Clássicos:")
print(f"  Trials por modelo: {OPTUNA_CLASSIC_N_TRIALS}")
print(f"  CV Splits: {OPTUNA_CLASSIC_CV_SPLITS}-fold stratified")
print(f"  Métrica de otimização: PR-AUC (ideal para classes desbalanceadas)")

⚙️ Configuração Optuna para Modelos Clássicos:
  Trials por modelo: 50
  CV Splits: 5-fold stratified
  Métrica de otimização: PR-AUC (ideal para classes desbalanceadas)


### 5.1 _Random Forest_

In [6]:
def compute_class_weight(class_weight, *, classes, y):
    """Estimate class weights for unbalanced datasets.

    Parameters
    ----------
    class_weight : dict, "balanced" or None
        If "balanced", class weights will be given by
        `n_samples / (n_classes * np.bincount(y))`.
        If a dictionary is given, keys are classes and values are corresponding class
        weights.
        If `None` is given, the class weights will be uniform.

    classes : ndarray
        Array of the classes occurring in the data, as given by
        `np.unique(y_org)` with `y_org` the original class labels.

    y : array-like of shape (n_samples,)
        Array of original class labels per sample.

    Returns
    -------
    class_weight_vect : ndarray of shape (n_classes,)
        Array with `class_weight_vect[i]` the weight for i-th class.

    References
    ----------
    The "balanced" heuristic is inspired by
    Logistic Regression in Rare Events Data, King, Zen, 2001.

    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.utils.class_weight import compute_class_weight
    >>> y = [1, 1, 1, 1, 0, 0]
    >>> compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
    array([1.5 , 0.75])
    """

    if set(y) - set(classes):
        raise ValueError("classes should include all valid labels that can be in y")
    if class_weight is None or len(class_weight) == 0:
        # uniform class weights
        weight = np.ones(classes.shape[0], dtype=np.float64, order="C")
    elif class_weight == "balanced":
        # Find the weight of each class as present in y.
        le = LabelEncoder()
        y_ind = le.fit_transform(y)
        if not all(np.isin(classes, le.classes_)):
            raise ValueError("classes should have valid labels that are in y")

        recip_freq = len(y) / (len(le.classes_) * np.bincount(y_ind).astype(np.float64))
        weight = recip_freq[le.transform(classes)]
    else:
        # user-defined dictionary
        weight = np.ones(classes.shape[0], dtype=np.float64, order="C")
        unweighted_classes = []
        for i, c in enumerate(classes):
            if c in class_weight:
                weight[i] = class_weight[c]
            else:
                unweighted_classes.append(c)

        n_weighted_classes = len(classes) - len(unweighted_classes)
        if unweighted_classes and n_weighted_classes != len(class_weight):
            unweighted_classes_user_friendly_str = np.array(unweighted_classes).tolist()
            raise ValueError(
                f"The classes, {unweighted_classes_user_friendly_str}, are not in"
                " class_weight"
            )

    return weight

cw_rf = compute_class_weight('balanced', classes=np.unique(y_dev_ml), y=y_dev_ml)
class_weight_rf = dict(enumerate(cw_rf))

In [7]:
# 1️⃣ Otimização Random Forest com Optuna
def objective_random_forest(trial):
    """
    Função objetivo para otimizar Random Forest usando PR-AUC como métrica
    """
    # Sugerir hiperparâmetros
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=50)
    max_depth = trial.suggest_int('max_depth', 10, 50, step=5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.5, 0.7])
    
    # Calcular class weight
    cw = compute_class_weight('balanced', classes=np.unique(y_dev_ml), y=y_dev_ml)
    class_weight = dict(enumerate(cw))
    
    # Modelo
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        class_weight=class_weight,
        random_state=RANDOM_SEED,
        n_jobs=-1,
        verbose=0
    )
    
    # Validação cruzada estratificada
    skf = StratifiedKFold(n_splits=OPTUNA_CLASSIC_CV_SPLITS, shuffle=True, random_state=RANDOM_SEED)
    pr_auc_scores = []
    
    for train_idx, val_idx in skf.split(X_dev_ml, y_dev_ml):
        X_train, X_val = X_dev_ml[train_idx], X_dev_ml[val_idx]
        y_train, y_val = y_dev_ml[train_idx], y_dev_ml[val_idx]
        
        model.fit(X_train, y_train)
        y_pred_prob = model.predict_proba(X_val)[:, 1]
        
        pr_auc = average_precision_score(y_val, y_pred_prob)
        pr_auc_scores.append(pr_auc)
    
    return np.mean(pr_auc_scores)

print(f"\n{'='*70}")
print(f"🔄 OTIMIZANDO RANDOM FOREST COM OPTUNA")
print(f"{'='*70}")

study_rf = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=RANDOM_SEED, multivariate=True),
    study_name='random_forest_optimization'
)

study_rf.optimize(objective_random_forest, n_trials=OPTUNA_CLASSIC_N_TRIALS, show_progress_bar=True)

print(f"\n✓ Otimização concluída!")
print(f"  Melhor PR-AUC (CV): {study_rf.best_value:.4f}")
print(f"\nMelhores hiperparâmetros Random Forest:")
for key, value in study_rf.best_params.items():
    print(f"  {key}: {value}")

/home/denysderlian/anaconda3/envs/NNenv/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-11-19 01:05:53,620] A new study created in memory with name: random_forest_optimization



🔄 OTIMIZANDO RANDOM FOREST COM OPTUNA


Best trial: 0. Best value: 0.984343:   2%|▏         | 1/50 [00:02<02:09,  2.63s/it]

[I 2025-11-19 01:05:56,254] Trial 0 finished with value: 0.9843426344896932 and parameters: {'n_estimators': 250, 'max_depth': 50, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 0.7}. Best is trial 0 with value: 0.9843426344896932.


Best trial: 0. Best value: 0.984343:   4%|▍         | 2/50 [00:06<02:44,  3.43s/it]

...

[I 2025-11-19 01:08:37,525] Trial 47 finished with value: 0.9956663848971541 and parameters: {'n_estimators': 200, 'max_depth': 45, 'min_samples_split': 17, 'min_samples_leaf': 1, 'max_features': 0.7}. Best is trial 45 with value: 0.9999999999999998.


Best trial: 45. Best value: 1:  98%|█████████▊| 49/50 [02:46<00:03,  3.37s/it]

[I 2025-11-19 01:08:40,032] Trial 48 finished with value: 0.9978021978021976 and parameters: {'n_estimators': 250, 'max_depth': 45, 'min_samples_split': 12, 'min_samples_leaf': 1, 'max_features': 0.5}. Best is trial 45 with value: 0.9999999999999998.


Best trial: 45. Best value: 1: 100%|██████████| 50/50 [02:51<00:00,  3.42s/it]

[I 2025-11-19 01:08:44,862] Trial 49 finished with value: 0.9989010989010989 and parameters: {'n_estimators': 500, 'max_depth': 30, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 0.5}. Best is trial 45 with value: 0.9999999999999998.

✓ Otimização concluída!
  Melhor PR-AUC (CV): 1.0000

Melhores hiperparâmetros Random Forest:
  n_estimators: 400
  max_depth: 40
  min_samples_split: 18
  min_samples_leaf: 1
  max_features: 0.5


In [8]:
# Combinar todo o dataset (Dev + Test)
X_all_ml = np.vstack([X_dev_ml, X_test_ml])
y_all_ml = np.hstack([y_dev_ml, y_test_ml])

# Treinar o modelo Random Forest final com os melhores hiperparâmetros e avaliar com o stratified K-Fold
best_rf_params = study_rf.best_params
best_rf_model = RandomForestClassifier(
    n_estimators=best_rf_params['n_estimators'],
    max_depth=best_rf_params['max_depth'],
    min_samples_split=best_rf_params['min_samples_split'],
    min_samples_leaf=best_rf_params['min_samples_leaf'],
    max_features=best_rf_params['max_features'],
    class_weight=class_weight_rf,
    random_state=RANDOM_SEED,
    n_jobs=-1,
    verbose=0
)

skf = StratifiedKFold(n_splits=OPTUNA_CLASSIC_CV_SPLITS, shuffle=True, random_state=RANDOM_SEED)
pr_auc_scores_final_RF, roc_auc_scores_final_RF, f1_scores_final_RF, precision_scores_final_RF, \
recall_scores_final_RF, accuracy_scores_final_RF, cm_final_RF = [], [], [], [], [], [], []
for train_idx, val_idx in skf.split(X_all_ml, y_all_ml):
    X_train, X_val = X_all_ml[train_idx], X_all_ml[val_idx]
    y_train, y_val = y_all_ml[train_idx], y_all_ml[val_idx]
    
    best_rf_model.fit(X_train, y_train)
    y_pred_prob = best_rf_model.predict_proba(X_val)[:, 1]
    
    pr_auc = average_precision_score(y_val, y_pred_prob)
    roc_auc = roc_auc_score(y_val, y_pred_prob)
    f1 = f1_score(y_val, (y_pred_prob >= 0.5).astype(int), zero_division=0)
    precision = precision_score(y_val, (y_pred_prob >= 0.5).astype(int), zero_division=0)
    recall = recall_score(y_val, (y_pred_prob >= 0.5).astype(int), zero_division=0)
    accuracy = accuracy_score(y_val, (y_pred_prob >= 0.5).astype(int))
    cm = confusion_matrix(y_val, (y_pred_prob >= 0.5).astype(int))

    pr_auc_scores_final_RF.append(pr_auc)
    roc_auc_scores_final_RF.append(roc_auc)
    f1_scores_final_RF.append(f1)
    precision_scores_final_RF.append(precision)
    recall_scores_final_RF.append(recall)
    accuracy_scores_final_RF.append(accuracy)
    cm_final_RF.append(cm)

mean_pr_auc_final_RF = np.mean(pr_auc_scores_final_RF)
mean_roc_auc_final_RF = np.mean(roc_auc_scores_final_RF)
mean_f1_final_RF = np.mean(f1_scores_final_RF)
mean_precision_final_RF = np.mean(precision_scores_final_RF)
mean_recall_final_RF = np.mean(recall_scores_final_RF)
mean_accuracy_final_RF = np.mean(accuracy_scores_final_RF)
total_cm_RF = np.sum(cm_final_RF, axis=0)

print(f"\n✓ Avaliação final Random Forest (CV {OPTUNA_CLASSIC_CV_SPLITS}-fold): PR-AUC = {mean_pr_auc_final_RF:.4f}, ROC-AUC = {mean_roc_auc_final_RF:.4f}, F1 = {mean_f1_final_RF:.4f}, Precision = {mean_precision_final_RF:.4f}, Recall = {mean_recall_final_RF:.4f}, Accuracy = {mean_accuracy_final_RF:.4f}")
print(f"Confusion Matrix acumulada:\n{total_cm_RF}")


✓ Avaliação final Random Forest (CV 5-fold): PR-AUC = 0.9993, ROC-AUC = 0.9999, F1 = 0.9681, Precision = 1.0000, Recall = 0.9397, Accuracy = 0.9950
Confusion Matrix acumulada:
[[918   0]
 [  5  77]]


### 5.2 _SVM_

In [9]:
# 2️⃣ Otimização SVM com Optuna
def objective_svm(trial):
    """
    Função objetivo para otimizar SVM usando PR-AUC como métrica
    """
    # Sugerir hiperparâmetros
    C = trial.suggest_float('C', 0.01, 100, log=True)
    kernel = trial.suggest_categorical('kernel', ['rbf', 'poly', 'sigmoid'])
    
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)
    else:
        degree = 3  # valor padrão
    
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    
    # Calcular class weight
    cw = compute_class_weight('balanced', classes=np.unique(y_dev_ml), y=y_dev_ml)
    class_weight = dict(enumerate(cw))
    
    # Modelo
    model = SVC(
        C=C,
        kernel=kernel,
        degree=degree,
        gamma=gamma,
        class_weight=class_weight,
        probability=True,
        random_state=RANDOM_SEED,
        max_iter=2000
    )
    
    # Validação cruzada estratificada
    skf = StratifiedKFold(n_splits=OPTUNA_CLASSIC_CV_SPLITS, shuffle=True, random_state=RANDOM_SEED)
    pr_auc_scores = []
    
    for train_idx, val_idx in skf.split(X_dev_ml, y_dev_ml):
        X_train, X_val = X_dev_ml[train_idx], X_dev_ml[val_idx]
        y_train, y_val = y_dev_ml[train_idx], y_dev_ml[val_idx]
        
        model.fit(X_train, y_train)
        y_pred_prob = model.predict_proba(X_val)[:, 1]
        
        pr_auc = average_precision_score(y_val, y_pred_prob)
        pr_auc_scores.append(pr_auc)
    
    return np.mean(pr_auc_scores)

print(f"\n{'='*70}")
print(f"🔄 OTIMIZANDO SVM COM OPTUNA")
print(f"{'='*70}")

study_svm = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=RANDOM_SEED, multivariate=True),
    study_name='svm_optimization'
)

study_svm.optimize(objective_svm, n_trials=OPTUNA_CLASSIC_N_TRIALS, show_progress_bar=True)

print(f"\n✓ Otimização concluída!")
print(f"  Melhor PR-AUC (CV): {study_svm.best_value:.4f}")
print(f"\nMelhores hiperparâmetros SVM:")
for key, value in study_svm.best_params.items():
    print(f"  {key}: {value}")

/home/denysderlian/anaconda3/envs/NNenv/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-11-19 01:08:48,944] A new study created in memory with name: svm_optimization



🔄 OTIMIZANDO SVM COM OPTUNA


Best trial: 0. Best value: 0.69992:   2%|▏         | 1/50 [00:00<00:11,  4.32it/s]

[I 2025-11-19 01:08:49,176] Trial 0 finished with value: 0.6999203568474335 and parameters: {'C': 0.31489116479568624, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.6999203568474335.


Best trial: 2. Best value: 0.73783:   6%|▌         | 3/50 [00:00<00:08,  5.48it/s]

...

[I 2025-11-19 01:08:57,523] Trial 46 finished with value: 0.7452208170274434 and parameters: {'C': 2.0904154295218462, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 25 with value: 0.7671926091085121.
[I 2025-11-19 01:08:57,720] Trial 47 finished with value: 0.7490782111102868 and parameters: {'C': 0.6035116226619337, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 25 with value: 0.7671926091085121.


Best trial: 25. Best value: 0.767193:  98%|█████████▊| 49/50 [00:08<00:00,  5.84it/s]

[I 2025-11-19 01:08:57,887] Trial 48 finished with value: 0.7539402542176717 and parameters: {'C': 1.1340037347896497, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 25 with value: 0.7671926091085121.


Best trial: 25. Best value: 0.767193: 100%|██████████| 50/50 [00:09<00:00,  5.39it/s]

[I 2025-11-19 01:08:58,225] Trial 49 finished with value: 0.6028439927115569 and parameters: {'C': 0.4323397492584797, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 25 with value: 0.7671926091085121.

✓ Otimização concluída!
  Melhor PR-AUC (CV): 0.7672

Melhores hiperparâmetros SVM:
  C: 0.8806857709923557
  kernel: rbf
  gamma: auto


In [10]:
cw_svm = compute_class_weight('balanced', classes=np.unique(y_dev_ml), y=y_dev_ml)
class_weight_svm = dict(enumerate(cw_svm))

# Treinar o modelo SVM final com os melhores hiperparâmetros e avaliar com o stratified K-Fold
best_svm_params = study_svm.best_params
best_svm_model = SVC(
    C=best_svm_params['C'],
    kernel=best_svm_params['kernel'],
    degree=best_svm_params.get('degree', 3),
    gamma=best_svm_params['gamma'],
    class_weight=class_weight_svm,
    probability=True,
    random_state=RANDOM_SEED,
    max_iter=2000
)

skf = StratifiedKFold(n_splits=OPTUNA_CLASSIC_CV_SPLITS, shuffle=True, random_state=RANDOM_SEED)
pr_auc_scores_final_SVM, roc_auc_scores_final_SVM, \
f1_scores_final_SVM, precision_scores_final_SVM, recall_scores_final_SVM, \
accuracy_scores_final_SVM, cm_final_SVM = [], [], [], [], [], [], []
for train_idx, val_idx in skf.split(X_all_ml, y_all_ml):
    X_train, X_val = X_all_ml[train_idx], X_all_ml[val_idx]
    y_train, y_val = y_all_ml[train_idx], y_all_ml[val_idx]
    
    best_svm_model.fit(X_train, y_train)
    y_pred_prob = best_svm_model.predict_proba(X_val)[:, 1]

    pr_auc = average_precision_score(y_val, y_pred_prob)
    roc_auc = roc_auc_score(y_val, y_pred_prob)
    f1 = f1_score(y_val, (y_pred_prob >= 0.5).astype(int), zero_division=0)
    precision = precision_score(y_val, (y_pred_prob >= 0.5).astype(int), zero_division=0)
    recall = recall_score(y_val, (y_pred_prob >= 0.5).astype(int), zero_division=0)
    accuracy = accuracy_score(y_val, (y_pred_prob >= 0.5).astype(int))
    cm = confusion_matrix(y_val, (y_pred_prob >= 0.5).astype(int))

    pr_auc_scores_final_SVM.append(pr_auc)
    roc_auc_scores_final_SVM.append(roc_auc)
    f1_scores_final_SVM.append(f1)
    precision_scores_final_SVM.append(precision)
    recall_scores_final_SVM.append(recall)
    accuracy_scores_final_SVM.append(accuracy)
    cm_final_SVM.append(cm)

mean_pr_auc_final_SVM = np.mean(pr_auc_scores_final_SVM)
mean_roc_auc_final_SVM = np.mean(roc_auc_scores_final_SVM)
mean_f1_final_SVM = np.mean(f1_scores_final_SVM)
mean_precision_final_SVM = np.mean(precision_scores_final_SVM)
mean_recall_final_SVM = np.mean(recall_scores_final_SVM)
mean_accuracy_final_SVM = np.mean(accuracy_scores_final_SVM)
total_cm_SVM = sum(cm_final_SVM)

print(f"\n✓ Avaliação final SVM (CV {OPTUNA_CLASSIC_CV_SPLITS}-fold): PR-AUC = {mean_pr_auc_final_SVM:.4f}, ROC-AUC = {mean_roc_auc_final_SVM:.4f}, F1 = {mean_f1_final_SVM:.4f}, Precision = {mean_precision_final_SVM:.4f}, Recall = {mean_recall_final_SVM:.4f}, Accuracy = {mean_accuracy_final_SVM:.4f}")
print(f"Confusion Matrix acumulada:\n{total_cm_SVM}")


✓ Avaliação final SVM (CV 5-fold): PR-AUC = 0.7422, ROC-AUC = 0.9580, F1 = 0.6127, Precision = 0.7236, Recall = 0.5515, Accuracy = 0.9440
Confusion Matrix acumulada:
[[899  19]
 [ 37  45]]


### 5.3 _XGBoost_

In [11]:
# 3️⃣ Otimização XGBoost com Optuna
def objective_xgboost(trial):
    """
    Função objetivo para otimizar XGBoost usando PR-AUC como métrica
    """
    # Calcular scale_pos_weight para desbalanceamento
    scale_pos_weight = (y_dev_ml == 0).sum() / (y_dev_ml == 1).sum()
    
    # Sugerir hiperparâmetros
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=50)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    gamma = trial.suggest_float('gamma', 0.0, 5.0)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 10.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 10.0)
    
    # Modelo
    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        min_child_weight=min_child_weight,
        gamma=gamma,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        scale_pos_weight=scale_pos_weight,
        random_state=RANDOM_SEED,
        n_jobs=-1,
        verbosity=0,
        eval_metric='logloss'
    )
    
    # Validação cruzada estratificada
    skf = StratifiedKFold(n_splits=OPTUNA_CLASSIC_CV_SPLITS, shuffle=True, random_state=RANDOM_SEED)
    pr_auc_scores = []
    
    for train_idx, val_idx in skf.split(X_dev_ml, y_dev_ml):
        X_train, X_val = X_dev_ml[train_idx], X_dev_ml[val_idx]
        y_train, y_val = y_dev_ml[train_idx], y_dev_ml[val_idx]
        
        model.fit(X_train, y_train)
        y_pred_prob = model.predict_proba(X_val)[:, 1]
        
        pr_auc = average_precision_score(y_val, y_pred_prob)
        pr_auc_scores.append(pr_auc)
    
    return np.mean(pr_auc_scores)

print(f"\n{'='*70}")
print(f"🔄 OTIMIZANDO XGBOOST COM OPTUNA")
print(f"{'='*70}")

study_xgb = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=RANDOM_SEED, multivariate=True),
    study_name='xgboost_optimization'
)

study_xgb.optimize(objective_xgboost, n_trials=OPTUNA_CLASSIC_N_TRIALS, show_progress_bar=True)

print(f"\n✓ Otimização concluída!")
print(f"  Melhor PR-AUC (CV): {study_xgb.best_value:.4f}")
print(f"\nMelhores hiperparâmetros XGBoost:")
for key, value in study_xgb.best_params.items():
    print(f"  {key}: {value}")

/home/denysderlian/anaconda3/envs/NNenv/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-11-19 01:08:58,574] A new study created in memory with name: xgboost_optimization



🔄 OTIMIZANDO XGBOOST COM OPTUNA


Best trial: 0. Best value: 0.963585:   2%|▏         | 1/50 [01:22<1:07:09, 82.24s/it]

[I 2025-11-19 01:10:20,812] Trial 0 finished with value: 0.9635851105416322 and parameters: {'n_estimators': 250, 'max_depth': 10, 'learning_rate': 0.1205712628744377, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'min_child_weight': 2, 'gamma': 0.2904180608409973, 'reg_alpha': 8.661761457749352, 'reg_lambda': 6.011150117432088}. Best is trial 0 with value: 0.9635851105416322.


Best trial: 1. Best value: 0.969647:   4%|▍         | 2/50 [01:52<41:28, 51.85s/it]  

...

[I 2025-11-19 01:11:30,604] Trial 47 finished with value: 0.9639181108746326 and parameters: {'n_estimators': 150, 'max_depth': 6, 'learning_rate': 0.013488971777329106, 'subsample': 0.936575875018079, 'colsample_bytree': 0.7964967008336085, 'min_child_weight': 3, 'gamma': 3.598307240496079, 'reg_alpha': 2.2674600348108878, 'reg_lambda': 6.238123587198323}. Best is trial 31 with value: 0.9854645354645355.


Best trial: 31. Best value: 0.985465:  98%|█████████▊| 49/50 [02:32<00:00,  1.87it/s]

[I 2025-11-19 01:11:30,963] Trial 48 finished with value: 0.9819939958231814 and parameters: {'n_estimators': 150, 'max_depth': 8, 'learning_rate': 0.04780032705694896, 'subsample': 0.9854926857104741, 'colsample_bytree': 0.9796522495209655, 'min_child_weight': 1, 'gamma': 3.1033546697435215, 'reg_alpha': 3.5033292462591077, 'reg_lambda': 5.005132318044005}. Best is trial 31 with value: 0.9854645354645355.


Best trial: 31. Best value: 0.985465: 100%|██████████| 50/50 [02:33<00:00,  3.06s/it]

[I 2025-11-19 01:11:31,670] Trial 49 finished with value: 0.9671671465789113 and parameters: {'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.010061485273576346, 'subsample': 0.8809487866559865, 'colsample_bytree': 0.9879972258007352, 'min_child_weight': 1, 'gamma': 2.566718027378215, 'reg_alpha': 2.086324068021754, 'reg_lambda': 6.64068967878981}. Best is trial 31 with value: 0.9854645354645355.

✓ Otimização concluída!
  Melhor PR-AUC (CV): 0.9855

Melhores hiperparâmetros XGBoost:
  n_estimators: 200
  max_depth: 9
  learning_rate: 0.08460717673292172
  subsample: 0.9148411603186509
  colsample_bytree: 0.8874705513948635
  min_child_weight: 2
  gamma: 0.4556961022635173
  reg_alpha: 1.3642495812843543
  reg_lambda: 0.03182598407445281


In [12]:
scale_pos_weight = (y_dev_ml == 0).sum() / (y_dev_ml == 1).sum()

# Treinar o modelo XGBoost final com os melhores hiperparâmetros e avaliar com o stratified K-Fold
best_xgb_params = study_xgb.best_params
best_xgb_model = XGBClassifier(
    n_estimators=best_xgb_params['n_estimators'],
    max_depth=best_xgb_params['max_depth'],
    learning_rate=best_xgb_params['learning_rate'],
    subsample=best_xgb_params['subsample'],
    colsample_bytree=best_xgb_params['colsample_bytree'],
    min_child_weight=best_xgb_params['min_child_weight'],
    gamma=best_xgb_params['gamma'],
    reg_alpha=best_xgb_params['reg_alpha'],
    reg_lambda=best_xgb_params['reg_lambda'],
    scale_pos_weight=scale_pos_weight,
    random_state=RANDOM_SEED,
    n_jobs=-1,
    verbosity=0,
    eval_metric='logloss'
)

skf = StratifiedKFold(n_splits=OPTUNA_CLASSIC_CV_SPLITS, shuffle=True, random_state=RANDOM_SEED)
pr_auc_scores_final_XGB, roc_auc_scores_final_XGB, f1_scores_final_XGB, precision_scores_final_XGB, \
recall_scores_final_XGB, accuracy_scores_final_XGB, cm_final_XGB = [], [], [], [], [], [], []
for train_idx, val_idx in skf.split(X_all_ml, y_all_ml):
    X_train, X_val = X_all_ml[train_idx], X_all_ml[val_idx]
    y_train, y_val = y_all_ml[train_idx], y_all_ml[val_idx]
    
    best_xgb_model.fit(X_train, y_train)
    y_pred_prob = best_xgb_model.predict_proba(X_val)[:, 1]

    pr_auc = average_precision_score(y_val, y_pred_prob)
    roc_auc = roc_auc_score(y_val, y_pred_prob)
    f1 = f1_score(y_val, (y_pred_prob >= 0.5).astype(int), zero_division=0)
    precision = precision_score(y_val, (y_pred_prob >= 0.5).astype(int), zero_division=0)
    recall = recall_score(y_val, (y_pred_prob >= 0.5).astype(int), zero_division=0)
    accuracy = accuracy_score(y_val, (y_pred_prob >= 0.5).astype(int))
    cm = confusion_matrix(y_val, (y_pred_prob >= 0.5).astype(int))

    pr_auc_scores_final_XGB.append(pr_auc)
    roc_auc_scores_final_XGB.append(roc_auc)
    f1_scores_final_XGB.append(f1)
    precision_scores_final_XGB.append(precision)
    recall_scores_final_XGB.append(recall)
    accuracy_scores_final_XGB.append(accuracy)
    cm_final_XGB.append(cm)

mean_pr_auc_final_XGB = np.mean(pr_auc_scores_final_XGB)
mean_roc_auc_final_XGB = np.mean(roc_auc_scores_final_XGB)
mean_f1_final_XGB = np.mean(f1_scores_final_XGB)
mean_precision_final_XGB = np.mean(precision_scores_final_XGB)
mean_recall_final_XGB = np.mean(recall_scores_final_XGB)
mean_accuracy_final_XGB = np.mean(accuracy_scores_final_XGB)
total_cm_XGB = np.sum(cm_final_XGB, axis=0)

print(f"\n✓ Avaliação final XGBoost (CV {OPTUNA_CLASSIC_CV_SPLITS}-fold): PR-AUC = {mean_pr_auc_final_XGB:.4f}, ROC-AUC = {mean_roc_auc_final_XGB:.4f}, F1 = {mean_f1_final_XGB:.4f}, Precision = {mean_precision_final_XGB:.4f}, Recall = {mean_recall_final_XGB:.4f}, Accuracy = {mean_accuracy_final_XGB:.4f}")
print(f"Confusion Matrix acumulada:\n{total_cm_XGB}")


✓ Avaliação final XGBoost (CV 5-fold): PR-AUC = 0.9911, ROC-AUC = 0.9986, F1 = 0.9806, Precision = 1.0000, Recall = 0.9632, Accuracy = 0.9970
Confusion Matrix acumulada:
[[918   0]
 [  3  79]]


### 5.4 Comparação entre os métodos clássicos


In [13]:
print(f"\n{'='*70}")
print(f"RESUMO COMPARATIVO DOS MODELOS CLÁSSICOS")
print(f"{'='*70}")

models_summary = pd.DataFrame({
    'Model': ['Random Forest', 'SVM', 'XGBoost'],
    'PR-AUC (CV)': [mean_pr_auc_final_RF, mean_pr_auc_final_SVM, mean_pr_auc_final_XGB],
    'ROC-AUC (CV)': [mean_roc_auc_final_RF, mean_roc_auc_final_SVM, mean_roc_auc_final_XGB],
    'F1-Score (CV)': [mean_f1_final_RF, mean_f1_final_SVM, mean_f1_final_XGB],
    'Precision (CV)': [mean_precision_final_RF, mean_precision_final_SVM, mean_precision_final_XGB],
    'Recall (CV)': [mean_recall_final_RF, mean_recall_final_SVM, mean_recall_final_XGB],
    'Accuracy (CV)': [mean_accuracy_final_RF, mean_accuracy_final_SVM, mean_accuracy_final_XGB]
})

print(models_summary.to_string(index=False))

# Imprimir as matrizes de confusão finais
print(f"\nMatrizes de Confusão Finais:")
print(f"\nRandom Forest:\n{total_cm_RF}")
print(f"\nSVM:\n{total_cm_SVM}")
print(f"\nXGBoost:\n{total_cm_XGB}")



RESUMO COMPARATIVO DOS MODELOS CLÁSSICOS
        Model  PR-AUC (CV)  ROC-AUC (CV)  F1-Score (CV)  Precision (CV)  Recall (CV)  Accuracy (CV)
Random Forest     0.999346      0.999936       0.968106        1.000000     0.939706          0.995
          SVM     0.742226      0.957998       0.612738        0.723571     0.551471          0.944
      XGBoost     0.991076      0.998592       0.980606        1.000000     0.963235          0.997

Matrizes de Confusão Finais:

Random Forest:
[[918   0]
 [  5  77]]

SVM:
[[899  19]
 [ 37  45]]

XGBoost:
[[918   0]
 [  3  79]]
